<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/ICaRL---Lucia/ICaRLMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [0]:
import os
import logging
import sys

In [2]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone -b ICaRL---Lucia https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 893 (delta 108), reused 6 (delta 2), pack-reused 719
Receiving objects: 100% (893/893), 574.72 KiB | 838.00 KiB/s, done.
Resolving deltas: 100% (560/560), done.


# Import packages

In [0]:

from DatasetCIFAR.data_set import Dataset 
from DatasetCIFAR import ResNet
from DatasetCIFAR import utils
from DatasetCIFAR import params
from DatasetCIFAR import ICaRLModel
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np

from torchvision import transforms
from torch.utils.data import Subset, DataLoader
from torch.nn import functional as F
import random
random.seed(params.SEED)

# Define Datasets

In [0]:
train_transformer = transforms.Compose([transforms.RandomCrop(size = 32, padding=4),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transformer = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
trainDS = Dataset(train=True, transform = train_transformer)
testDS = Dataset(train=False, transform = test_transformer)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [6]:
train_splits = trainDS.splits
test_splits = testDS.splits
print(train_splits)
print(test_splits)

[[94.0, 63.0, 74.0, 21.0, 35.0, 56.0, 91.0, 96.0, 87.0, 48.0], [68.0, 80.0, 22.0, 37.0, 60.0, 97.0, 51.0, 62.0, 92.0, 76.0], [75.0, 89.0, 23.0, 99.0, 39.0, 66.0, 54.0, 69.0, 84.0, 61.0], [85.0, 24.0, 98.0, 41.0, 73.0, 58.0, 78.0, 77.0, 70.0, 49.0], [65.0, 88.0, 36.0, 93.0, 45.0, 10.0, 90.0, 17.0, 32.0, 59.0], [83.0, 43.0, 53.0, 11.0, 86.0, 19.0, 38.0, 30.0, 40.0, 50.0], [57.0, 81.0, 12.0, 95.0, 25.0, 47.0, 34.0, 52.0, 44.0, 72.0], [46.0, 79.0, 20.0, 28.0, 5.0, 71.0, 8.0, 18.0, 33.0, 15.0], [55.0, 29.0, 64.0, 31.0, 67.0, 7.0, 13.0, 14.0, 42.0, 6.0], [82.0, 2.0, 27.0, 16.0, 26.0, 3.0, 4.0, 1.0, 9.0, 0.0]]
[[94.0, 63.0, 74.0, 21.0, 35.0, 56.0, 91.0, 96.0, 87.0, 48.0], [68.0, 80.0, 22.0, 37.0, 60.0, 97.0, 51.0, 62.0, 92.0, 76.0], [75.0, 89.0, 23.0, 99.0, 39.0, 66.0, 54.0, 69.0, 84.0, 61.0], [85.0, 24.0, 98.0, 41.0, 73.0, 58.0, 78.0, 77.0, 70.0, 49.0], [65.0, 88.0, 36.0, 93.0, 45.0, 10.0, 90.0, 17.0, 32.0, 59.0], [83.0, 43.0, 53.0, 11.0, 86.0, 19.0, 38.0, 30.0, 40.0, 50.0], [57.0, 81.0, 12.

# Define Network

In [0]:
ICaRL = ICaRLModel.ICaRLStruct(dataset = trainDS)

# Train phase

In [0]:
pars_tasks = []
test_indexes = []

for task in range(0, 100, params.TASK_SIZE):
  pars_tasks.insert(task, 0)

In [0]:
for task in range(0, 100, params.TASK_SIZE):

  train_indexes = trainDS.__getIndexesGroups__(task)
  test_indexes = test_indexes + testDS.__getIndexesGroups__(task)

  train_dataset = Subset(trainDS, train_indexes)
  test_dataset = Subset(testDS, test_indexes)

  train_loader = DataLoader( train_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE)
  test_loader = DataLoader( test_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE )

  ICaRL.updateRep(task, train_dataset, train_splits[int(task/10)], train_transformer)

  m = ICaRL.k / (task + params.TASK_SIZE) #task + params.TASK_SIZE represents the total amount of classes know at each step
  print('m: ', m)
  ICaRL.reduceExemplars(int(m))

  col = []
  for i,x in enumerate( test_splits[ :int(task/10) + 1]) : 
    v = np.array(x)
    col = np.concatenate( (col,v), axis = None)
    col = col.astype(int)

  for y in train_splits[int(task/10)]:
    images = []
    for i, el in enumerate(train_dataset):
      if( el[1] == y):
        images.append(train_indexes[i])
    ICaRL.generateExemplars(images, int(m), int(y) )


  total = 0.0
  running_corrects = 0.0
  for img, lbl, _ in train_loader:
      img = img.float().to(params.DEVICE)
      preds = ICaRL.classify(img, col)
      labels = utils.mapFunction(lbl, col)
      #print("preds: ", preds.data)
      #print("mapped labels: ", labels)
      #print("labels: ", lbl)
      total += len(lbl)
      running_corrects += torch.sum(preds == labels.data).data.item()
      #print(running_corrects)
  accuracy = float(running_corrects/total)
  print(f'task: {task}', f'train accuracy = {accuracy}')


  total = 0.0
  running_corrects = 0.0
  for img, lbl, _ in test_loader:
      img = img.float().to(params.DEVICE)
      preds = ICaRL.classify(img, col)
      labels = utils.mapFunction(lbl, col)
      total += len(lbl)
      running_corrects += torch.sum(preds == labels.data).data.item()
  accuracy = float(running_corrects/total)
  print(f'task: {task}', f'test accuracy = {accuracy}')

Files already downloaded and verified
task = 0 
Task:  0  epoch:  0  loss:  0.055286820977926254
Task:  0  epoch:  1  loss:  0.051184576004743576
m:  200.0
len exemplars[ 94 ] =  200
len exemplars[ 63 ] =  200
len exemplars[ 74 ] =  200
len exemplars[ 21 ] =  200
len exemplars[ 35 ] =  200
len exemplars[ 56 ] =  200
len exemplars[ 91 ] =  200
len exemplars[ 96 ] =  200
len exemplars[ 87 ] =  200
len exemplars[ 48 ] =  200
col =  [94 63 74 21 35 56 91 96 87 48]
preds:  tensor([0, 6, 4, 6, 4, 0, 3, 9, 0, 3, 0, 8, 0, 3, 9, 4, 9, 3, 9, 8, 0, 6, 9, 0,
        0, 0, 0, 0, 6, 0, 9, 9, 8, 6, 0, 0, 0, 3, 0, 6, 9, 0, 0, 9, 4, 5, 0, 4,
        4, 0, 3, 9, 3, 9, 0, 0, 9, 9, 3, 6, 4, 4, 9, 3, 0, 6, 3, 4, 0, 6, 6, 3,
        0, 0, 0, 3, 6, 9, 9, 3, 0, 6, 4, 0, 9, 3, 0, 4, 4, 4, 6, 3, 9, 9, 9, 6,
        3, 7, 4, 6, 0, 8, 4, 3, 4, 9, 3, 8, 8, 3, 0, 9, 9, 0, 5, 9, 6, 9, 3, 9,
        3, 4, 0, 0, 6, 3, 0, 0], device='cuda:0')
mapped labels:  tensor([7, 2, 8, 3, 5, 8, 4, 4, 5, 0, 7, 8, 8, 4, 1, 1, 2, 9,

In [0]:
total = 0.0
correct = 0.0
for img, lbl, _ in train_loader:
    img = img.float().to(params.DEVICE)
    preds = ICaRL.classify(img, train_splits[: int(task/10) + 1])
    total += lbl.size(0)
    correct += (preds.data.cpu() == lbl).sum()

In [0]:
a = 10
b = 9
print(f'task: {a}', f'accuracy = {b}')